In [243]:
import numpy as np
import pandas as pd
import cv2

det = pd.read_csv('det/det.txt', sep=',', header=None)

columns = [
    "frame",
    "id",
    "bb_left",
    "bb_top",
    "bb_width",
    "bb_height",
    "conf",
    "x",
    "y",
    "z"
]
det.columns = columns
det

,frame,id,bb_left,bb_top,bb_width,bb_height,conf,x,y,z
0,1,-1,1689,385,146.620,332.710,67.567,-1,-1,-1
1,1,-1,1303,503,61.514,139.590,29.439,-1,-1,-1
2,1,-1,1258,569,40.123,91.049,19.601,-1,-1,-1
3,1,-1,31,525,113.370,257.270,17.013,-1,-1,-1
4,1,-1,1800,483,94.660,214.810,11.949,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...
4897,525,-1,1479,580,28.302,64.224,36.629,-1,-1,-1
4898,525,-1,1806,521,36.723,83.333,23.327,-1,-1,-1
4899,525,-1,1820,381,123.420,280.060,14.567,-1,-1,-1
4900,525,-1,1655,425,94.660,214.810,12.803,-1,-1,-1


In [244]:
# filter bboxes with conf
conf_threshold = 30.0
det = det[det.conf >= conf_threshold]
det

,frame,id,bb_left,bb_top,bb_width,bb_height,conf,x,y,z
0,1,-1,1689,385,146.620,332.710,67.567,-1,-1,-1
5,2,-1,1689,385,146.620,332.710,66.725,-1,-1,-1
6,2,-1,1312,503,61.514,139.590,36.614,-1,-1,-1
11,3,-1,1742,513,94.660,214.810,64.803,-1,-1,-1
12,3,-1,1664,418,135.420,307.290,31.198,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...
4886,524,-1,1281,447,87.838,199.320,33.172,-1,-1,-1
4894,525,-1,724,283,226.740,514.530,52.497,-1,-1,-1
4895,525,-1,1602,403,113.370,257.270,46.469,-1,-1,-1
4896,525,-1,1281,447,87.838,199.320,40.393,-1,-1,-1


In [245]:
det.drop(columns=["x", "y", "z"], inplace=True)

C:\Users\Tom\AppData\Local\Temp\ipykernel_14292\1947031159.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  det.drop(columns=["x", "y", "z"], inplace=True)


In [246]:
def get_frames(df):
    frames = {}
    for frame in df["frame"].unique():
        current = []
        for _, row in df[df["frame"] == frame].iterrows():
            current.append(row[2:6].values)
        frames[frame] = current
    return frames

det_frames = get_frames(det)
det_frames[list(det_frames.keys())[0]]

[array([1689.  ,  385.  ,  146.62,  332.71])]

In [247]:
# compute jaccard index
def jaccard_index(a, b):
    a_left, a_top, a_width, a_height = a
    b_left, b_top, b_width, b_height = b
    a_right = a_left + a_width
    a_bottom = a_top + a_height
    b_right = b_left + b_width
    b_bottom = b_top + b_height
    xA = max(a_left, b_left)
    yA = max(a_top, b_top)
    xB = min(a_right, b_right)
    yB = min(a_bottom, b_bottom)
    interArea = max(0, xB - xA) * max(0, yB - yA)
    aArea = a_width * a_height
    bArea = b_width * b_height
    return interArea / float(aArea + bArea - interArea)

In [248]:
# compute jaccard index for all frames between frame n and frame n - 1
def jaccard_index_frames(det_frames):
    jaccard_index_frames = {}
    for frame in det_frames:
        if frame == 1:
            continue
        jaccard_index_frame = np.zeros((len(det_frames[frame]), len(det_frames[frame - 1])))
        for i, a in enumerate(det_frames[frame]):
            for j, b in enumerate(det_frames[frame - 1]):
                jaccard_index_frame[i][j] = jaccard_index(a, b)
        jaccard_index_frames[frame] = jaccard_index_frame
    return jaccard_index_frames

jaccard_index_frames = jaccard_index_frames(det_frames)
jaccard_index_frames[list(jaccard_index_frames.keys())[0]]

array([[1.],
       [0.]])

In [249]:
# associate the detections to tracks in a greedy manner using IoU threshold sigma_iou
sigma_iou = 0.7
def associate_detections_to_tracks(jaccard_index_frames):
    tracks = {}
    jaccard_values = {}
    for frame in jaccard_index_frames.keys():
        tracks[frame] = []
        jaccard_values[frame] = []
        for jaccard_index_frame in jaccard_index_frames[frame]:
            if len(jaccard_index_frame) == 0:
                tracks[frame].append(-1)
                continue
            max_index = np.argmax(jaccard_index_frame)
            if jaccard_index_frame[max_index] >= sigma_iou:
                tracks[frame].append(max_index)
                jaccard_values[frame].append(jaccard_index_frame[max_index])
            else:
                tracks[frame].append(-1)
                jaccard_values[frame].append(-1)
    return tracks, jaccard_values

tracks, jaccard_values = associate_detections_to_tracks(jaccard_index_frames)
for i in range(2, 6):
    print(f"Frame Tracks{i}: {tracks[i]}")
    print(f"Frame jaccard_values {i}: {jaccard_values[i]}")

Frame Tracks2: [0, -1]
Frame jaccard_values 2: [0.9999999999999988, -1]
Frame Tracks3: [-1, -1, 1]
Frame jaccard_values 3: [-1, -1, 0.999999999999997]
Frame Tracks4: [-1, -1]
Frame jaccard_values 4: [-1, -1]
Frame Tracks5: [0, -1]
Frame jaccard_values 5: [0.738903394255874, -1]


In [250]:
# track management
def track_management(tracks, jaccard_values):
    """
    Track management
     Each object can be assigned to only one trajectory (ID)
     Create and update lists for matches, unmatched detections and unmatched tracks
     Matches: IoU ≥ sigma_iou -> matched track
     Unmatched tracks -> delete track
     Unmatched detection -> create new tracks
    """
    bboxes_for_each_frame = {}
    for current_frame in list(tracks.keys())[1:]:
        previous_frame = current_frame - 1
        p_tracks = np.array(tracks[previous_frame])
        c_tracks = np.array(tracks[current_frame])
        # sort
        jaccard_dict = {}
        for i, t in enumerate(c_tracks):
            jaccard_dict[t] = jaccard_values[current_frame][i]
        p_tracks = np.sort(p_tracks)
        c_tracks = np.sort(c_tracks)
        # remove -1
        p_tracks = p_tracks[p_tracks >= 0]
        c_tracks = c_tracks[c_tracks >= 0]
        # uniques only
        p_tracks = np.unique(p_tracks)
        c_tracks = np.unique(c_tracks)
        # print("----------------------")
        # print(f"Previous Tracks Frame {previous_frame}: {p_tracks}")
        # print(f"Current Tracks Frame {current_frame}: {c_tracks}")
        matches = []
        unmatched_tracks = [] # objects that have disappeared
        unmatched_detections = [] # new appearances of objects

        for i, track in enumerate(p_tracks):
            if track not in c_tracks:
                unmatched_tracks.append(track)

        for i, track in enumerate(c_tracks):
            if track in p_tracks:
                matches.append(track)
            else:
                unmatched_detections.append(track)
        # print(f"Matches Frame {current_frame}: {matches}")
        # print(f"Unmatched Tracks Frame {current_frame}: {unmatched_tracks}")
        # print(f"Unmatched Detections Frame {current_frame}: {unmatched_detections}")
        bboxes_for_each_frame[current_frame] = {
            "matches": matches,
            "unmatched_tracks": unmatched_tracks,
            "unmatched_detections": unmatched_detections,
            "jaccard_dict": jaccard_dict
        }
    return bboxes_for_each_frame

bboxes_for_each_frame = track_management(tracks, jaccard_values)
bboxes_for_each_frame

{3: {'matches': [],
  'unmatched_tracks': [0],
  'unmatched_detections': [1],
  'jaccard_dict': {-1: -1, 1: 0.999999999999997}},
 4: {'matches': [],
  'unmatched_tracks': [1],
  'unmatched_detections': [],
  'jaccard_dict': {-1: -1}},
 5: {'matches': [],
  'unmatched_tracks': [],
  'unmatched_detections': [0],
  'jaccard_dict': {0: 0.738903394255874, -1: -1}},
 6: {'matches': [],
  'unmatched_tracks': [0],
  'unmatched_detections': [1],
  'jaccard_dict': {-1: -1, 1: 0.8387423559486619}},
 7: {'matches': [1],
  'unmatched_tracks': [],
  'unmatched_detections': [0],
  'jaccard_dict': {0: 0.820549494820784, 1: 0.9999999999999948}},
 8: {'matches': [0],
  'unmatched_tracks': [1],
  'unmatched_detections': [],
  'jaccard_dict': {0: 0.9999999999999992, -1: -1}},
 9: {'matches': [0],
  'unmatched_tracks': [],
  'unmatched_detections': [1],
  'jaccard_dict': {0: 0.820549494820784, 1: 0.7372720200282675, -1: -1}},
 10: {'matches': [0, 1],
  'unmatched_tracks': [],
  'unmatched_detections': [],


In [251]:
import cv2
import matplotlib.pyplot as plt
import os
from tqdm import tqdm

"""
Develop an interface for tracking results check to see if the tracker properly keeps track of objects by
associating the correct IDs in the video stream
 Draw rectangular bounding box around the detected object in images
 Draw attributed ID to each tracked objects
 Draw the trajectory (tracking path ) in an image
"""

def draw_bboxes(frame, bboxes, color):
    for bbox in bboxes:
        x, y, w, h = bbox.astype(int)
        cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)

def draw_ids(frame, bboxes, jaccard_dict, ids, color):
    for bbox, id in zip(bboxes, ids):
        x, y, w, h = bbox.astype(int)
        cv2.putText(frame, str(id), (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 2, color, 2)
        if jaccard_dict is not None:
            jac = jaccard_dict[id]
            cv2.putText(frame, str(round(jac, 2)), (x, y + h + 10), cv2.FONT_HERSHEY_SIMPLEX, 2, color, 2)

frame_imgs = []
for frame in tqdm(list(bboxes_for_each_frame.keys())):
    img = cv2.imread(f"img1/{frame:06d}.jpg")
    matches = bboxes_for_each_frame[frame]["matches"]
    unmatched_detections = bboxes_for_each_frame[frame]["unmatched_detections"]
    unmatched_tracks = bboxes_for_each_frame[frame]["unmatched_tracks"]
    jaccard_values = bboxes_for_each_frame[frame]["jaccard_dict"]

    bbox_matches = []
    bbox_unmatched_detections = []
    bbox_unmatched_tracks = []
    for i, bbox in enumerate(det_frames[frame]):
        if i in matches:
            bbox_matches.append(bbox)
        elif i in unmatched_detections:
            bbox_unmatched_detections.append(bbox)
        elif i in unmatched_tracks:
            bbox_unmatched_tracks.append(bbox)
    
    draw_bboxes(img, bbox_matches, (255, 255, 255))
    draw_ids(img, bbox_matches, jaccard_values, matches, (255, 255, 255))

    draw_bboxes(img, bbox_unmatched_detections, (0, 255, 0))
    draw_ids(img, bbox_unmatched_detections, jaccard_values, unmatched_detections, (0, 255, 0))

    draw_bboxes(img, bbox_unmatched_tracks, (0, 0, 255))
    draw_ids(img, bbox_unmatched_tracks, None, unmatched_tracks, (0, 0, 255))
    frame_imgs.append(img)

100%|██████████| 523/523 [00:09<00:00, 54.17it/s]


In [252]:
# pop a window with a video stream from the frame_imgs
fps = 30
def show_video(frame_imgs):
    for frame in frame_imgs:
        cv2.imshow("video", frame)
        if cv2.waitKey(1000 // fps) & 0xFF == ord('q'):
            break
    cv2.destroyAllWindows()

show_video(frame_imgs)